In [11]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import joblib
from sklearn import metrics
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression



In [12]:
df = pd.read_pickle('../../../data.2020-11-09.scenarioJunction.pkl.gz')


In [13]:
df

,team_red_2_0,name_red_2_0,index_red_2_0,kind_red_2_0,move_red_2_0,load_red_2_0,hp_red_2_0,hp_max_red_2_0,int_atk_red_2_0,int-def_red_2_0,...,weapon_AT_blue_1_2,weapon_MT_blue_1_2,weapon_GR_blue_1_2,weapon_SM_blue_1_2,weapon_SR_blue_1_2,winner,meta_scenario,meta_p_red,meta_p_blue,meta_seed
0,red,rTank3,0,2,7,1,1,1,6,0,...,4,2,2,0,0,red,scenarioJunction,GreedyAgent,GreedyAgent,837183820
1,red,rTank3,0,2,7,1,1,1,6,0,...,4,2,2,0,0,red,scenarioJunction,GreedyAgent,GreedyAgent,837183820
2,red,rTank3,0,2,7,1,1,1,6,0,...,4,2,2,0,0,red,scenarioJunction,GreedyAgent,GreedyAgent,837183820
3,red,rTank3,0,2,7,1,1,1,6,0,...,4,2,2,0,0,red,scenarioJunction,GreedyAgent,GreedyAgent,837183820
4,red,rTank3,0,2,7,1,1,1,6,0,...,4,2,2,0,0,red,scenarioJunction,GreedyAgent,GreedyAgent,837183820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,red,rTank3,0,2,7,1,1,1,6,1,...,4,5,2,0,0,red,scenarioJunctionExo,GreedyAgent,RandomAgent,375130710
20,red,rTank3,0,2,7,1,1,1,6,1,...,4,5,2,0,0,red,scenarioJunctionExo,GreedyAgent,RandomAgent,375130710
21,red,rTank3,0,2,7,1,1,1,6,1,...,4,5,2,0,0,red,scenarioJunctionExo,GreedyAgent,RandomAgent,375130710
22,red,rTank3,0,2,7,1,1,1,5,1,...,4,5,2,0,0,red,scenarioJunctionExo,GreedyAgent,RandomAgent,375130710


In [14]:
def pipeline_save(df):
    X = df.drop('winner', axis=1)
    y = df['winner']    
    #categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c=df.select_dtypes(include=['object']).drop(['winner'], axis=1).columns
    preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), c)])
    rf = Pipeline(steps=[('preprocessor', preprocessor),('classifier', RandomForestClassifier())])
    rf.fit(X, y)
    return(rf)


In [15]:
def pipelineClassifier(df):   
    X = df.drop('winner', axis=1)
    y = df['winner']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c=df.select_dtypes(include=['object']).drop([['winner', 'meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed']], axis=1,errors="ignore").columns
    preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, c)])
    classifiers = [
        KNeighborsClassifier(3),
        SVC(kernel="rbf", C=0.025, probability=True),
        NuSVC(probability=True),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        AdaBoostClassifier(),
        GradientBoostingClassifier()
        ]
    for classifier in classifiers:
        pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', classifier)])
        pipe.fit(X_train, y_train)
        y_pred=pipe.predict(X_test)
        print(classifier)
        print("model score: %.3f" % pipe.score(X_test, y_test))
        print("model accuracy:",metrics.accuracy_score(y_test,y_pred))
        #print("model f1 score:",metrics.f1_score(y_test,y_pred))

In [ ]:
def pipelineClassifierCross(df):   
    X = df.drop('winner', axis=1)
    y = df['winner']
    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c=df.select_dtypes(include=['object']).drop(['winner'], axis=1).columns
    preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, c)])
    classifiers = [
        KNeighborsClassifier(3),
        #SVC(kernel="rbf", C=0.025, probability=True),
        NuSVC(probability=True),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        AdaBoostClassifier(),
        GradientBoostingClassifier()
        ]
    for classifier in classifiers:
        pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', classifier)])
        scores = cross_val_score(pipe, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
        print(classifier.__class__.__name__)
        print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))
     







In [ ]:
pipelineClassifierCross(df)

In [1]:
model=joblib.load('model.joblib')

NameError: name 'joblib' is not defined

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import pandas as pd
from sklearn.naive_bayes import GaussianNB

import joblib


def pipelineClassifier(df, name):
    X = df.drop(['winner', 'meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed'], axis=1, errors="ignore")
    y = df['winner']
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c = df.select_dtypes(include=['object']).drop(['winner', 'meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed'],
                                                  axis=1, errors="ignore").columns
    preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, c)])
    classifiers = [
        KNeighborsClassifier(3),
        # SVC(kernel="rbf", C=0.025, probability=True),
        # NuSVC(probability=True),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        AdaBoostClassifier(),
        GradientBoostingClassifier(),
        GaussianNB(),
    ]
    for classifier in classifiers:
        pipe = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', classifier)])
        pipe.fit(X, y)
        file_name = f'{name}_{classifier.__class__.__name__}.joblib'
        joblib.dump(pipe, file_name)


if __name__ == '__main__':
    '''dataframes = {"BridgeHead": "../../../data.scenarioBridgeHead.pkl.gz",
                  "CrossingTheCity": "../../../data.scenarioCrossingTheCity.pkl.gz",
                  "Junction": "../../../data.scenarioJunction.pkl.gz",
                  "JunctionExo": "../../../data.scenarioJunctionExo.pkl.gz",
                  "Roadblock": "../../../data.scenarioRoadblock.pkl.gz",
                  "Test1v1": "../../../data.scenarioTest1v1.pkl.gz", "Test2v2": "../../../data.scenarioTest2v2.pkl.gz"}'''
    dataframes = {"Junction": "../../../data.2020-11-09.scenarioJunction.pkl.gz",
                  "JunctionExo": "../../../data.2020-11-09.scenarioJunctionExo.pkl.gz",
                  "Test1v1": "../../../data.2020-11-09.scenarioTest1v1.pkl.gz",
                  "Test2v2": "../../../data.2020-11-09.scenarioTest2v2.pkl.gz"}
    # pilots = ["BridgeHead", "CrossingTheCity", "Junction", "JunctionExo", "Roadblock", "Test1v1", "Test2v2"]
    pilots = ["Junction", "JunctionExo", "Test1v1", "Test2v2"]

    for p in pilots:
        df = pd.read_pickle(dataframes[p])
        '''df = df.loc[(((df.meta_p_red == "GreedyAgent") & (df.meta_p_blue == "GreedyAgent")) | (
                    (df.meta_p_red == "GreedyAgent") & (df.meta_p_blue == "RandomAgent")) | (
                                 (df.meta_p_red == "RandomAgent") & (df.meta_p_blue == "GreedyAgent")))]'''

        pipelineClassifier(df, p)


In [19]:
import os.path as op

In [21]:
dir_path = op.dirname(op.realpath("ciao"))


In [22]:
file="ciao"

In [23]:
op.join(dir_path, '..', '..', 'models', file)

'C:\\Users\\Nicol\\Documents\\Master\\SecondoProgetto\\newtechnowar-webui\\code\\notebooks\\..\\..\\models\\ciao'